In [1]:
import yfinance as yf
import pandas as pd
import numpy
import math
import altair as alt
import time
import json
import datetime
import sqlite3
import multiprocessing  
import random
from multiprocessing import Pool
import defs
import time
import datetime
import requests
import re

import requests_cache
#requests_cache.install_cache(cache_name="cache_cache", backend='sqlite')

def Logging(val):
    with open("log.txt") as file: # Use file to refer to the file object
        file.write(val)


#missing
#fees
#dividends
#stocksplits - what happens in  a stock split(are old shares amount converted to new share amount)
#confirm values, see if value is between open and close of that day
#drop columns when amount is 0

def return_print(msg="",payload=None,status="INFO"):
    
    payload={}
    if payload is None:
        payload=payload

    payload['status']=status
    payload['msg']=msg
    
    print(json.dumps(payload))

from random_proxies import random_proxy


#TODO
#fees 
#dividends

In [2]:
filename = 'FinancePositions_Loader.csv'


f = open(filename, "r")


count_comma=0
count_semicolon=0
for i in range(2):
    line = f.readline()
    count_comma=count_comma+line.count(",")
    count_semicolon=count_semicolon+line.count(";")


f.close()

return_print("Importing Transactions")

sep=","
if(count_semicolon>count_comma):
    sep=";"


transactions = pd.read_csv(filename,sep=sep)

#get the most earliest date
df=pd.DataFrame()
df['Date']=transactions['When']
startdate=pd.to_datetime(df['Date']).sort_values()[0].strftime("%Y-%m-%d")

#get all use tickers distinct
stonks=pd.unique(transactions['Ticker']).tolist()

return_print("Transactions Loaded")

transactions



_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Ticker,What,Total Cost,Amount,When,Cost per Share,Currency
0,Exxon Mobil Corporation,T,buy,215.8,5,07/08/2020,43.16,USD
1,Exxon Mobil Corporation,MO,buy,215.8,5,07/08/2020,43.16,USD
2,Exxon Mobil Corporation,DAL,buy,215.8,5,07/08/2020,43.16,USD
3,Exxon Mobil Corporation,AA,buy,215.8,5,07/08/2020,43.16,USD
4,Exxon Mobil Corporation,AXP,buy,215.8,5,07/08/2020,43.16,USD
...,...,...,...,...,...,...,...,...
109706,Exxon Mobil Corporation,G5N.SG,buy,215.8,NaN,07/08/2020,43.16,USD
109707,Exxon Mobil Corporation,N1H.AX,buy,215.8,NaN,07/08/2020,43.16,USD
109708,Exxon Mobil Corporation,1K2.BE,buy,215.8,NaN,07/08/2020,43.16,USD
109709,Exxon Mobil Corporation,2RM.BE,buy,215.8,NaN,07/08/2020,43.16,USD


In [3]:
# #fetches information about the stock (namely currency)

# return_print("Fetching Information on the stocks, this may take a minute...")

# #fetch from db, if not exist, fetch from internet
# stonks_info={}

# stonks_missing=[]


# #fetch ticker data akready obtained and saved locally
# con = sqlite3.connect('example.db')

# # #for every stock check if in local db
# for stonk in stonks:

#     print("fetching",stonk)
#     cur = con.cursor()

#     t = (stonk,)
#     cur.execute('SELECT * FROM stocks_info WHERE ticker=?', t)

#     s_info=cur.fetchone() 

#     #store in dict
#     if(s_info is not None):
#         stonks_info[stonk] = s_info
#         print(stonk)
#     else:
#         stonks_missing.append(stonk)

In [4]:
#len(stonks_missing)
print(stonks)


, 'G5N.DU', 'G5N.F', 'G5N.BE', 'G5N.SG', 'N1H.AX', '1K2.BE', '2RM.BE', '2RM.F']


In [7]:
# cur.execute("select count(*) from stocks_info")

# cur.fetchall()

In [18]:

def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.

    try:
        #inf=defs.FetchTickerInfo(i)

        print("Fetched")

        cony = sqlite3.connect('example.db')
        cursy = cony.cursor()
        print("Inserting")

        x = re.findall("\"currency\":\"(.*?)\"", result[1].decode('UTF-8'))[0]

        cursy.execute('INSERT INTO stocks_currency VALUES (?,?)', ( result[0], x))
        cony.commit()
        print(result[0],x)
    except Exception as e:
        print(e)
        print("Whoopsie")


#multithread load of stocks not already obtained 
infos=[]
if __name__ == '__main__':
    pool = Pool(16)
    #infos = pool.map(defs.FetchTickerInfo, stonks_missing)

    count=0
    for i in stonks:
        count=count+1
        print(count)
        pool.apply_async(defs.FetchTickerInfo3, args = (i, ), callback = log_result)
    print("Thread Started")
    pool.close()
    pool.join()
    #print(result_list)

#get current format
curtime=datetime.datetime.now().isoformat()

stonks_to_add = []

for tick, inf in zip(stonks_missing,infos):


    stonks_info[tick] = inf
    stonks_to_add.append((tick, json.dumps(inf),curtime))

#save to avoid future big times
cur.executemany('INSERT INTO stocks_info VALUES (?,?,?)', stonks_to_add)


#T16 - 13
#T4 - 26 
print(time.time()-start)

con.commit()
con.close()


SD
Fetched
Inserting
PAR USD
Fetched
Inserting
CHH USD
Fetched
Inserting
EE USD


In [3]:

#donwload all data
data = yf.download(tickers=stonks,start=startdate,interval='1d',group_by='column',prepos=False,threads=True)

pairs = yf.download(tickers="EURUSD=X",start=startdate,interval='1d',group_by='column',prepos=False,threads=True)

#counts entries without values
empty_stocks  =data['Close'].columns[data['Close'].count()==0].to_list()

if len(empty_stocks)>0:
    payload= {}
    payload['stocks'] = empty_stocks
    return_print("The Following Tickers are Invalid:"+" ".join(empty_stocks)+"\n, tickers must be set accordingly to  https://finance.yahoo.com/",payload,status='ERROR')

return_print("Stock Values Loaded")


[                       0%                       ]  96 of 106294 completed

KeyboardInterrupt: 

In [127]:
stonkdict={}


targetcurrency='EUR'

superdf =pd.DataFrame()
print(superdf.empty)

i=0


for stonk in stonks:  
    
    print(stonk)
    
    #get transactions from csv
    singlestock_transactions = transactions[transactions['Ticker']==stonk]

    #use index to create df
    index = data.index
    df=pd.DataFrame(index=data.index)

    
    #extract price
    df['Price']= data['Close'][stonk]
    #initialize amount
    df["SharesOwned"]=0
    #initialize total value
    df['TotalValue']=0   

    df['InvestedValue']=0   

    #set ticker
    df["Ticker"]=stonk
    
    #fetch currency
    df["Currency"]=json.loads(stonks_info['DIOD'][1])['currency']


    for idx, row in singlestock_transactions.iterrows():

        multiplier = 1
        if(row['What']=='buy'):
            multiplier=1
        elif(row['What']=='sell'):
            multiplier=-1
        else:
            raise Exception("Transaction \"row['What']\" is not recognized")

        #print(row['Amount'])


        #CHECK IF IT GETS APPLIED TO STARTING ARRAY, FIND AWAY TO DO THIS AT THE START
        #CONVERTING INTO AMOUNT MAKES it be adimensional, and doenst matter what currency it was bought in
        try:
            if(math.isnan(float(row['Amount']))):
                row['Amount']=row['Total Cost']/row['Cost per Share']
        except:
            row['Amount']=row['Total Cost']/row['Cost per Share']


        #df[row['When']:][stonk+" Amount"] = df[row['When']:][stonk+" Amount"]=row['Amount']

        #df[row['When']:,stonk+" Amount"] = df[row['When']:,stonk+" Amount"]+row['Amount']

        #print(float(row['Amount']))
        
        if pd.to_datetime("now")<pd.to_datetime(row['When']):
            print(row['When'],"<- This date is in the future, and will not be considere")
            continue


        
        #print(float(row['Amount'])*multiplier)
        #set amount to the amount
        #display(row['When'])
        #display(df.loc[row['When']:])
        
        
        
        
        #change the amount from whtn to forward,  multiplier determinex if its a  buy of a sell
        df.loc[row['When']:,'SharesOwned'] = df.loc[row['When']:,'SharesOwned'] + (float(row['Amount'])*multiplier)

        #money put in
        df.loc[row['When']:,'InvestedValue'] = df.loc[row['When']:,'InvestedValue'] + (float(row['Total Cost'])*multiplier)
        
        
        
        

        

    i=i+1
        
    df.loc[:,'TotalValue']= df["Price"]*df['SharesOwned']
    df.loc[:,'EarnedValue']= df.loc[:,'TotalValue']-df.loc[:,'InvestedValue']
    
    
        
    
    
    
    
    #delete rows with no data
    #
    df.interpolate(inplace=True)
    df.dropna(inplace=True,how='any')
    
    
    #display(df)
    #display(df)
    
    superdf= superdf.append(df)
    #display(df)
    #display(superdf)
    #print("YIKES")

    




True
XOM
WMT
WDC
V
11/02/2021 <- This date is in the future, and will not be considere
TSLA
TEAM
T
PG
PEP
ORCL
11/03/2021 <- This date is in the future, and will not be considere
NVDA
MSFT
MMM
11/02/2021 <- This date is in the future, and will not be considere
MCD
KO
INTC
IBM
GOOG
FB
DIS
CRM
BRK-B
BABA
AAL
AMZN
AMD
AAPL
DNNGY
05/01/2021 <- This date is in the future, and will not be considere
05/03/2021 <- This date is in the future, and will not be considere
JNJ
PLTR
ZM
12/03/2021 <- This date is in the future, and will not be considere
DIOD
NNND.F
IWDA.AS
12/02/2021 <- This date is in the future, and will not be considere
ETH-USD
11/02/2021 <- This date is in the future, and will not be considere
BTC-USD
11/02/2021 <- This date is in the future, and will not be considere


'{"zip": "75024", "sector": "Technology", "fullTimeEmployees": 8306, "longBusinessSummary": "Diodes Incorporated designs, manufactures, and supplies application-specific standard products in the discrete, logic, and analog and mixed-signal semiconductor markets worldwide. It primarily focuses on low pin count semiconductor devices with one or more active and/or passive components. The company offers discrete semiconductor products, such as MOSFET, TVS, and performance Schottky rectifiers; GPP bridges and retifiers performance Schottky diodes; Zener and performance Zener diodes, including tight tolerance and low operating current type; standard, fast, super-fast, and ultra-fast recovery rectifiers; bridge rectifiers; switching diodes; small signal bipolar and prebiased transistors; other MOSFETs; thyristor surge protection devices; and transient voltage suppressors. It also provides analog products comprising power management devices consisting of AC-DC and DC-DC converters, USB power s

'USD'

In [128]:
superdf['CurrencyUSD']=0

In [129]:
fauxpair = pairs.loc[:]['Close']

In [130]:


hm = pd.merge(
    superdf,
    fauxpair,
    how="left",
    left_on='Date',
    right_on='Date',
    left_index=False,
    right_index=False,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

hm.loc[:]['Close']=hm.loc[:]['Close'].interpolate()




In [10]:
import sqlite3
con = sqlite3.connect('example.db')

cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks_info (ticker text, info text,dateinserted TEXT )''')

               # Save (commit) the changes
#cur.execute('DROP TABLE stocks_info')
#cur.execute('DROP TABLE stocks')

con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

None


In [132]:
from tinydb import TinyDB, Query
db = TinyDB('db.json')
db.insert({'type': 'apple', 'count': 7})

6

In [133]:
!pip install tinydb

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


#df.loc[row['When']:,'Amount'] = df.loc[row['When']:,'Amount'] + (float(row['Amount'])*multiplier)
hm.loc[:,hm['Currency']=='EUR'] = hm.loc[:,'Close']*hm.loc[:,'TotalValue']

In [134]:
#selecct column based on another column
hm.loc[hm['Currency']=='EUR',['Currency']]


#hm.loc[:,hm['Currency']=='EUR']['Close']
#hm[hm['Currency']=='EUR']['Currency']
#rectify us currency

#convert currency
hm.loc[:,'TotalValueUSD']=0
hm.loc[:,'TotalValueUSD']=hm.loc[:,'TotalValue']
hm.loc[hm['Currency']=='EUR','TotalValueUSD'] = hm[hm['Currency']=='EUR']['TotalValue']*hm[hm['Currency']=='EUR']['Close']

In [135]:
print(time.time()-startu)

8.346622943878174


In [136]:
#superdf[superdf['Ticker']=='XOM']
dfforaltair = superdf.loc["2019-10-20":].reset_index()
dfforaltair.interpolate(inplace=True)

alt.data_transformers.disable_max_rows()
alt.Chart(dfforaltair).mark_area().encode(
    x='Date',
    y='TotalValue',
    #color='Ticker:N',
    color=alt.Color('Ticker:N', legend=alt.Legend(columns=1,symbolLimit=300)),
).properties(
    width=1280,
    height=720
)

ipykernel_launcher:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.


alt.Chart(...)

In [137]:
for key in stonkdict:
    print(key)

702*0.50


df.loc['04/02/2020':, 'TSLAR'] = 2

df

In [ ]:
#df[df['1962-01-02':]][stonk+" Amount"]=df[df['1962-01-02':]][stonk+" Amount"]+row['Amount']



df

In [ ]:
!pip install --upgrade numpy
!pip install stocker